In [8]:
import langchain

from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
load_dotenv()

True

### 1. Langchain Basic

In [6]:
client = OpenAI()


In [7]:
prompt = "Can you tell me the total number of countries in Asia? Can you name the top 10 based on population"
print(client.predict(prompt))



As of 2021, there are 48 countries in Asia. The top 10 countries in Asia based on population are:

1. China
2. India
3. Indonesia
4. Pakistan
5. Bangladesh
6. Japan
7. Philippines
8. Vietnam
9. Iran
10. Turkey


### 2. Prompt Template

In [9]:
# 1st way 
prompt_template_name = PromptTemplate(
    input_variables = ['country'],
    template = 'Can you tell me the capital of {country}?'
)

In [10]:
prompt1 = prompt_template_name.format(country = 'India')
prompt2 = prompt_template_name.format(country = 'Romania')

In [12]:
client.predict(prompt1).strip()

'The capital of India is New Delhi.'

In [13]:
client.predict(prompt2).strip()

'The capital of Romania is Bucharest.'

In [14]:
# 2nd way
prompt_template_name_2 = PromptTemplate.from_template("What is a good name for a company that makes {products}?")

In [15]:
prompt3 = prompt_template_name_2.format(products = 'toys')

In [17]:
print(client.predict(prompt3).strip())

1. Playful Creations Co.
2. Toytopia Inc.
3. WonderWorks Toys
4. Imagination Factory
5. Funland Toys
6. Happyland Playthings
7. Toylandia Enterprises
8. Creativity Kids Co.
9. Joyful Jax Toys
10. Whimsy World Inc.
11. Dreamland Toys
12. Magic Makers Co.
13. The Toy Emporium
14. Playtime Innovations
15. Toy Kingdom Co.
16. Imagine and Play Co.
17. Sunny Side Toys
18. SparklePlay Co.
19. Toytopolis Inc.
20. WonderWorks Toy Co.


### 3. Agent